# 0 - Data to Snowflake


In [30]:
import json
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, sproc, col
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType, Variant
from snowflake.snowpark import functions as fn

import numpy as np
import pandas as pd

from snowflake.snowpark import version
import opendatasets as od
import pandas as pd
import snowflake.connector


In [31]:
# Gets the version
ctx = snowflake.connector.connect(
    user='maxellende',
    password='passwordSF22!',
    account='GVQJSJC-MP40342'
    )

connection_parameters = {
    "account": 'GVQJSJC-MP40342',
    "user": 'maxellende',
    "password": 'passwordSF22!',
    "role": "ACCOUNTADMIN"
}

session = Session.builder.configs(connection_parameters).create()


In [32]:
# Create a Virtual Warehouse
session.sql("CREATE OR REPLACE WAREHOUSE HOL_WH WITH WAREHOUSE_SIZE='MEDIUM' WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED';").collect()

# Create Databse
session.sql("CREATE OR REPLACE DATABASE HOL_DB").collect()

# od.download("https://www.kaggle.com/datasets/sgpjesus/bank-account-fraud-dataset-neurips-2022")
df = pd.read_csv("C:/Users/maxellende.julienne/Sujet de Stage - Paradigme Lakehouse/Project/bank-account-fraud-dataset-neurips-2022/Base.csv", sep=',', header=0, index_col=False)
print(df)

# write df in SF
session.write_pandas(df, table_name='CREDIT_RECORD', auto_create_table=True, overwrite=True)


        fraud_bool  income  name_email_similarity  prev_address_months_count  \
0                1     0.9               0.166828                         -1   
1                1     0.9               0.296286                         -1   
2                1     0.9               0.044985                         -1   
3                1     0.9               0.159511                         -1   
4                1     0.9               0.596414                         -1   
...            ...     ...                    ...                        ...   
999995           0     0.6               0.192631                         -1   
999996           0     0.8               0.322989                        148   
999997           0     0.8               0.879403                         -1   
999998           0     0.9               0.762112                         -1   
999999           0     0.2               0.697452                         -1   

        current_address_months_count  c

In [33]:
# Saving credentials to a JSON file for reuse
connection_parameters["database"] = 'HOL_DB'
connection_parameters["schema"] = 'PUBLIC'
connection_parameters["warehouse"] = 'HOL_WH'

with open('creds.json', 'w') as outfile:
        outfile.write(json.dumps(connection_parameters))


In [34]:
# Saving the installed packages and their versions as a Json file
import pkg_resources
env = dict(tuple(str(ws).split()) for ws in pkg_resources.working_set)

sp_udf_packages = ['snowflake-snowpark-python', 'pandas', 'scikit-learn', 'lightgbm', 'xgboost', 'cachetools', 'joblib', 'imbalanced-learn']

packages_version = dict((k , env[k]) for k in sp_udf_packages if k in  env )
with open('packages_version.json', 'w') as outfile:
    outfile.write(json.dumps(packages_version))

In [35]:
# Testing
df = session.table('CREDIT_RECORD')
df.show()
df.columns

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"fraud_bool"  |"income"            |"name_email_similarity"  |"prev_address_months_count"  |"current_address_months_count"  |"customer_age"  |"days_since_request"  |"intended_balcon_amount"  |"payment_type"  |"zip_count_4w"  |"velocity_6h"       |"velocity_24h"      |"velocity_4w"      |"bank

['"fraud_bool"',
 '"income"',
 '"name_email_similarity"',
 '"prev_address_months_count"',
 '"current_address_months_count"',
 '"customer_age"',
 '"days_since_request"',
 '"intended_balcon_amount"',
 '"payment_type"',
 '"zip_count_4w"',
 '"velocity_6h"',
 '"velocity_24h"',
 '"velocity_4w"',
 '"bank_branch_count_8w"',
 '"date_of_birth_distinct_emails_4w"',
 '"employment_status"',
 '"credit_risk_score"',
 '"email_is_free"',
 '"housing_status"',
 '"phone_home_valid"',
 '"phone_mobile_valid"',
 '"bank_months_count"',
 '"has_other_cards"',
 '"proposed_credit_limit"',
 '"foreign_request"',
 '"source"',
 '"session_length_in_minutes"',
 '"device_os"',
 '"keep_alive_session"',
 '"device_distinct_emails_8w"',
 '"device_fraud_count"',
 '"month"']

In [36]:
import snowflake.snowpark
import snowflake.snowpark.functions as F
def uppercase_all_columns(df: snowflake.snowpark.DataFrame) -> snowflake.snowpark.DataFrame:
    return df.select([F.col(column).as_(column.upper()) for column in df.columns])

In [37]:
df = uppercase_all_columns(df)
df.columns

['FRAUD_BOOL',
 'INCOME',
 'NAME_EMAIL_SIMILARITY',
 'PREV_ADDRESS_MONTHS_COUNT',
 'CURRENT_ADDRESS_MONTHS_COUNT',
 'CUSTOMER_AGE',
 'DAYS_SINCE_REQUEST',
 'INTENDED_BALCON_AMOUNT',
 'PAYMENT_TYPE',
 'ZIP_COUNT_4W',
 'VELOCITY_6H',
 'VELOCITY_24H',
 'VELOCITY_4W',
 'BANK_BRANCH_COUNT_8W',
 'DATE_OF_BIRTH_DISTINCT_EMAILS_4W',
 'EMPLOYMENT_STATUS',
 'CREDIT_RISK_SCORE',
 'EMAIL_IS_FREE',
 'HOUSING_STATUS',
 'PHONE_HOME_VALID',
 'PHONE_MOBILE_VALID',
 'BANK_MONTHS_COUNT',
 'HAS_OTHER_CARDS',
 'PROPOSED_CREDIT_LIMIT',
 'FOREIGN_REQUEST',
 'SOURCE',
 'SESSION_LENGTH_IN_MINUTES',
 'DEVICE_OS',
 'KEEP_ALIVE_SESSION',
 'DEVICE_DISTINCT_EMAILS_8W',
 'DEVICE_FRAUD_COUNT',
 'MONTH']

In [38]:
df.write.save_as_table(table_name='CREDIT_RECORD', mode='overwrite')
session.table('CREDIT_RECORD').limit(5).to_pandas()


,FRAUD_BOOL,INCOME,NAME_EMAIL_SIMILARITY,PREV_ADDRESS_MONTHS_COUNT,CURRENT_ADDRESS_MONTHS_COUNT,CUSTOMER_AGE,DAYS_SINCE_REQUEST,INTENDED_BALCON_AMOUNT,PAYMENT_TYPE,ZIP_COUNT_4W,...,HAS_OTHER_CARDS,PROPOSED_CREDIT_LIMIT,FOREIGN_REQUEST,SOURCE,SESSION_LENGTH_IN_MINUTES,DEVICE_OS,KEEP_ALIVE_SESSION,DEVICE_DISTINCT_EMAILS_8W,DEVICE_FRAUD_COUNT,MONTH
0,0,0.9,0.990175,28,27,40,0.007601,52.213070,AA,941,...,1,990.0,0,INTERNET,1.980750,macintosh,0,1,0,4
1,0,0.6,0.112168,101,4,50,6.435678,-0.377317,AC,994,...,0,200.0,0,INTERNET,33.644412,linux,0,1,0,4
2,0,0.6,0.371804,-1,112,20,0.026203,-1.072698,AB,2694,...,0,200.0,0,INTERNET,12.827417,windows,0,2,0,4
3,0,0.6,0.911222,-1,15,30,0.029809,-1.400195,AC,651,...,1,1000.0,0,INTERNET,7.264525,macintosh,0,1,0,4
4,0,0.9,0.298294,31,12,40,0.001457,-0.852282,AC,993,...,0,200.0,0,INTERNET,3.130317,other,0,1,0,4


In [39]:
session.close()